# Libraries

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Configurations

In [2]:
# List of water station names
water_station_names = ['Moharam Bek','El-Gabal El-Asfar','Rod El-Farag','Fustat','Mansoura',
                      'Abu Rawash','El-Sheikh Zayed','Aswan']

# Dictionary mapping each water station to its geographical coordinates (latitude, longitude)
water_stations_loc = {
    "El-Gabal El-Asfar": (30.1396, 31.3789),
    "Rod El-Farag": (30.1015, 31.2396),
    "Fustat": (30.0178, 31.2273),
    "Moharam Bek": (31.2140, 29.9245),
    "Mansoura": (31.0409, 31.3776),
    "Abu Rawash": (30.0476, 31.0316),
    "El-Sheikh Zayed": (30.0035, 30.9764),
    "Aswan": (24.0908, 32.8998),
}

# Dictionary representing the number of pumps available at each water station
water_stations_pump = {
    "El-Gabal El-Asfar": 15,
    "Rod El-Farag": 20,
    "Fustat": 10,
    "Moharam Bek": 17,
    "Mansoura": 7,
    "Abu Rawash": 8,
    "El-Sheikh Zayed": 13,
    "Aswan": 25,
}

# Generating a date range from '2024-01-01' to '2024-12-31' with hourly frequency
dates = pd.date_range(start='2024-01-01', end='2024-02-01', freq='h')

# List of possible pump operation statuses
list_of_operation_pumps = ['power on', 'standby', 'maintenance']


In [3]:
# Function to ensure the number of pumps on is within a specified range (bounded by min and max)
def boundering_pump_on(number_of_pumps_on, max, min=1):
    # If the number of pumps on exceeds the maximum limit, set it to the maximum
    if number_of_pumps_on > max:
        number_of_pumps_on = max
    # If the number of pumps on is below the minimum limit, set it to the minimum
    elif number_of_pumps_on < min:
        number_of_pumps_on = min
    # Return the adjusted number of pumps on
    return number_of_pumps_on


In [4]:
# Initialize an empty dictionary to store maintenance pump data for each water station
maintenance_pump_num_dict = {}

# Iterate over each water station name in the list of station names
for name in water_station_names:
    # Create an empty dictionary for each water station to store maintenance data for each month
    maintenance_pump_num_dict[name] = {}
    
    # Iterate over each month (1 to 12)
    for month in range(1, 13):
        # Assign a random list of pump numbers for maintenance
        # Randomly select between 0 and 2 pumps from a range of 1 to 12 for each month
        maintenance_pump_num_dict[name][month] = random.sample(range(1, 13), random.randint(0, 2))


# Generate Data

In [5]:
# Initialize empty lists to store pump and station data
data_pump = []
data_station = []
data_fact=[]
# Loop through each water station name
for station_name in water_station_names:
    # Generate a unique station ID using the hash of the station name
    station_id = hash(station_name)
    
    # Generate random pump parameters: head pressure, flow, power consumption
    pump_head_pressure = random.uniform(2, 5)  # Random pump head pressure in bar
    pump_flow = random.uniform(500, 700)  # Random pump flow in liters per hour
    power_fact = 0.8  # Constant power factor
    pump_cons = random.uniform(1000, 5000)  # Random power consumption in watts
    
    # Initialize an empty list to record the number of operating pumps per hour
    number_of_pumps_on = []
    
    # Retrieve the monthly maintenance data for the current station
    maintenance_pump_num_per_station = maintenance_pump_num_dict[station_name]
    
    # Add water station information to the data_station list
    data_station.append({
        'water_station_id': station_id,
        'Water_station_name': station_name,
        'lat': water_stations_loc[station_name][0],
        'lan': water_stations_loc[station_name][1],
        'Number_of_pumps': water_stations_pump[station_name]
    })
    
    # Loop through each date and time in the generated date range
    for date in dates:
        time_day = date.time()  # Extract the time component of the date
        day_date = date.date()  # Extract the date component
        
        # Determine the number of pumps under maintenance for the current month
        maintenance_pump_num = len(maintenance_pump_num_per_station[date.month])
        
        # Calculate the maximum number of pumps that can operate (subtract maintenance pumps)
        maximum_pump = water_stations_pump[station_name] - maintenance_pump_num
        
        # Determine the number of pumps operating at night (before 6 AM)
        if time_day < datetime.strptime("06:00:00", '%H:%M:%S').time():  # Night condition
            maximum_pump_on = int(maximum_pump * 0.3)  # Set the max pumps to 30% capacity
            
            # If previous data exists, adjust the number of operating pumps randomly
            if len(number_of_pumps_on) > 0:
                operate_pump = number_of_pumps_on[-1] + random.randint(-1, 1)
            else:
                operate_pump = random.randint(1, maximum_pump_on)
            
            # Ensure the number of pumps operating is within the valid bounds
            operate_pump = boundering_pump_on(operate_pump, maximum_pump_on)
        
        # For daytime operations (after 6 AM), adjust the operating pumps
        else:
            operate_pump = number_of_pumps_on[-1] + random.randint(-1, 1)
            operate_pump = boundering_pump_on(operate_pump, maximum_pump, int(maximum_pump * 0.3))
            
        # Calculate total input and output flow, and output pressure
        total_input_flow = operate_pump * random.uniform(500, 700)  # Input flow in liters/hour
        total_output_flow = total_input_flow * random.uniform(0.8, 0.95)  # Output flow
        total_output_pressure = operate_pump * random.uniform(1.5, 2.5)  # Output pressure
        
        # Append the number of pumps operating for this hour
        number_of_pumps_on.append(operate_pump)
        
        # Reset working hours for pumps at midnight (00:00:00)
        if time_day == datetime.strptime("00:00:00", '%H:%M:%S').time():
            working_hours = {pump: 0 for pump in range(water_stations_pump[station_name])}
        
        # Initialize pump status tracking
        inncrement = 0
        pump_status = {}
        
        # Loop through each pump at the station to determine its status
        for pump in range(water_stations_pump[station_name]):
            pump_id = f"{station_name}_{pump}"  # Create a unique pump ID
            
            # Check if the pump is under maintenance for the current month
            if pump in maintenance_pump_num_dict[station_name][date.month]:
                pump_status[pump] = "maintenance"
            # If not under maintenance, determine if it should be powered on or on standby
            elif inncrement < operate_pump:
                pump_status[pump] = "power on"
                inncrement += 1  # Track the number of pumps turned on
                working_hours[pump] += 1  # Increment the working hours for this pump
            else:
                pump_status[pump] = "standby"
            
            # Append pump operation data to the data_pump list
            data_pump.append({
                'Date': day_date,
                'time': time_day,
                'pump_ID': pump_id,
                'station_id': station_id,
                'pump_status': pump_status[pump],
                'pump_head_bar': pump_head_pressure,
                'pump_flow': pump_flow,
                'power_factor': power_fact,
                'power_consumption_watt': pump_cons,
                'Working_hour_per_day': working_hours[pump]
            })
            data_fact.append({
                'Date':day_date,
                'time':time_day,
                'pump_ID':pump_id,
                'station_id':station_id,
                'total_operating_pump':operate_pump,
                'total_maintenance_pump':maintenance_pump_num,
                'total_standby_pump':maximum_pump - operate_pump,
                'total_input_flow':total_input_flow,
                'total_output_flow':total_output_flow,
                'total_output_pressure':total_output_pressure,
                'tank_level':random.uniform(20,100)
            })

In [6]:
# Initialize an empty list to store maintenance data
maintenance_data = []

# List of available maintenance equipment
maintenance_equipment = [
    "Pipe Wrenches", "Pipe Cutters", "Pipe Threaders", "Hydraulic Pipe Benders", 
    "Valve Key", "Leak Detection Equipment", "Soldering Tools", "Pipe Inspection Cameras",
    "Drain Cleaning Machines", "Pipe Locators", "Excavators", "Shovels", "Trench Boxes", 
    "Safety Barriers", "Personal Protective Equipment (PPE)", "Water Pumps", "Pressure Gauges", 
    "Flow Meters", "Sealant and Adhesives", "Pipe Clamps", "Teflon Tape", "Repair Couplings", 
    "Patching Kits", "Torch", "Measuring Tapes"
]

# Loop through each unique date in the date range
for date in set(dates.date):
    # Randomly determine how many maintenance activities occur on this date (between 0 and 2)
    number_maintenance_date = random.randint(0, 2)
    
    # Loop through the determined number of maintenance activities for this date
    for number_maintenance in range(number_maintenance_date):
        # Create a unique maintenance ID using the date and maintenance number
        maintenance_id = f"{date}_{number_maintenance}M"
        
        # Randomly select an employee ID (between 100 and 200) for the maintenance activity
        employee_id = random.randint(100, 200)
        
        # Randomly select 3 pieces of equipment from the maintenance equipment list
        equipment_names = random.sample(maintenance_equipment, 3)
        
        # Loop through the selected equipment to record maintenance details
        for equipment_name in equipment_names:
            # Randomly generate a cost per hour for using the equipment (between 50 and 150)
            cost_per_hour = random.randint(50, 150)
            
            # Append the maintenance activity data to the maintenance_data list
            maintenance_data.append({
                'Date':date,
                'maintenance_id': maintenance_id,
                'employee_id': employee_id,
                'equipment_name': equipment_name,
                'cost_per_hour': cost_per_hour
            })


In [7]:
# Initialize an empty list to store customer complaint data
customer_data = []

# Define possible types of complaints
complaint_type = ['water shortage', 'water outage']

# Loop through each water station name in the list
for name_station in water_station_names:
    # Generate a unique station ID using the hash of the station name
    station_id = hash(name_station)
    
    # Loop through each unique date in the date range
    for date in set(dates.date):
        # Randomly determine how many customers made complaints on this date (between 0 and 10)
        customer_date = random.randint(0, 10)
        
        # Loop through the determined number of customers for the day
        for number_customer in range(customer_date):
            # Create a unique complaint ID using the date and customer number
            customer_id = f"{date}_{number_customer}C"
            
            # Randomly select a type of complaint from the complaint_type list
            complaint_type_choice = random.choice(complaint_type)
            
            # Randomly generate the time it took to resolve the complaint (between 1 and 8 hours)
            complaint_resolution_time = random.randint(1, 8)
            
            # Append the complaint data to the customer_data list
            customer_data.append({
                'complaint_id': customer_id,
                'water_station_id': station_id,
                'complaint_type': complaint_type_choice,
                'complaint_date': date,
                'complaint_resolution_time': complaint_resolution_time
            })


In [8]:
# Initialize an empty list to store leakage data
leakage_data = []

# Loop through each water station name in the list
for name in water_station_names:
    # Generate a unique station ID using the hash of the station name
    station_id = hash(name)
    
    # Loop through each unique date in the date range
    for date in set(dates.date):
        # Randomly determine how many leakages occur on this date (between 0 and 2)
        leakage_number = random.randint(0, 2)
        
        # Loop through the determined number of leakages for the day
        for id in range(leakage_number):
            # Create a unique leakage ID using the date and leakage number
            leakage_id = f"{date}_{id}L"
            
            # Randomly generate the latitude and longitude of the leakage location within a range
            leakage_lat = random.uniform(30.0178, 31.2140)
            leakage_lan = random.uniform(29.9245, 32.8998)
            
            # Randomly generate the time to resolve the leakage (between 1 and 10 hours)
            leakage_resolution_time = random.randint(1, 10)
            
            # Randomly generate the volume of the leakage (between 0.1 and 10 cubic meters)
            leakage_volume = random.uniform(0.1, 10)
            
            # Calculate the total water wasted based on leakage volume and resolution time
            leakage_water_wasted = leakage_volume * leakage_resolution_time
            
            # Append the leakage data to the leakage_data list
            leakage_data.append({
                'Date':date,
                'leakage_id': leakage_id,
                'water_station_id': station_id,
                'leakage_lat': leakage_lat,
                'leakage_lan': leakage_lan,
                'leakage_water_wasted': leakage_water_wasted
            })


# convert to data frame

In [9]:
# Convert the list of pump data (data_pump) into a DataFrame
pump_dim = pd.DataFrame(data_pump)

# Filter the DataFrame to include only rows where the pump status is 'maintenance'
maintenance_df = pump_dim[pump_dim['pump_status'] == 'maintenance']

# Group the maintenance data by 'pump_ID' and find the most recent maintenance date for each pump
last_maintenance_dates = maintenance_df.groupby('pump_ID')['Date'].max().reset_index()

# Rename the 'Date' column to 'last_maintenance_date' to indicate that it holds the last maintenance date
last_maintenance_dates.rename(columns={'Date': 'last_maintenance_date'}, inplace=True)

# Group the pump data by all columns except 'time' and 'Working_hour_per_day', 
# and keep the maximum value of 'Working_hour_per_day' for each unique combination
pump_dim = pd.DataFrame(pump_dim.groupby(list(pump_dim.columns.drop(['time', 'Working_hour_per_day']))).max('Working_hour_per_day')).reset_index()

# Merge the pump data with the last maintenance dates based on 'pump_ID', keeping all pumps in pump_dim (left join)
pump_dim = pd.merge(pump_dim, last_maintenance_dates, on='pump_ID', how='left')

# Save the resulting DataFrame as a CSV file named 'pump_dim.csv' without including the index
pump_dim.to_csv('pump_dim.csv', index=False)


In [10]:
# Convert the list of water station data (data_station) into a DataFrame
station_dim = pd.DataFrame(data_station)

# Save the resulting DataFrame as a CSV file named 'station_dim.csv' without including the index
station_dim.to_csv('station_dim.csv', index=False)


In [11]:
maintenance_dim=pd.DataFrame(maintenance_data)
maintenance_dim.to_csv('maintenance_dim.csv',index=False)

In [12]:
customer_dim=pd.DataFrame(customer_data)
customer_dim.to_csv('customer_dim.csv',index=False)

In [13]:
leakage_dim=pd.DataFrame(leakage_data)
leakage_dim.to_csv('leakage_dim.csv',index=False)

In [16]:
# Convert the list of fact data (data_fact) into a DataFrame
fact_table = pd.DataFrame(data_fact)
print(fact_table.shape)
print(fact_table.shape)
fact_table.drop_duplicates()
print(fact_table.shape)
# Save the resulting DataFrame as a CSV file named 'fact_table.csv' without including the index
fact_table.to_csv('fact_table.csv', index=False)


(85675, 11)
(85675, 11)
(85675, 11)
